# Swan Consulting

## Import Librairies and Data

In [1]:
# Data
import pandas as pd
import numpy as np

# Visualise
import matplotlib.pyplot as plt
import seaborn as sns

# Regex
import re


In [82]:
# Read in data
df = pd.read_csv('Swan_Project_Data.csv')

In [4]:
# Set option to display all columns
pd.set_option('display.max_columns', None)


In [54]:
# Look at first few rows of data
df.head(2)

,CustomerID,Count,Country,State,City,Zip Code,Lat Long,Latitude,Longitude,Gender,Senior Citizen,Partner,Dependents,Tenure Months,Phone Service,Multiple Lines,Internet Service,Online Security,Online Backup,Device Protection,Tech Support,Streaming TV,Streaming Movies,Contract,Paperless Billing,Payment Method,Monthly Charges,Total Charges,Churn Label,Churn Value,Churn Reason
0,3668-QPYBK,1,United States,California,Los Angeles,90003,"33.964131, -118.272783",33.964131,-118.272783,Male,No,No,No,2,Yes,No,DSL,Yes,Yes,No,No,No,No,Month-to-month,Yes,Mailed check,53.85,108.15,Yes,1,Competitor made better offer
1,9237-HQITU,1,United States,California,Los Angeles,90005,"34.059281, -118.30742",34.059281,-118.307420,Female,No,No,Yes,2,Yes,No,Fiber optic,No,No,No,No,No,No,Month-to-month,Yes,Electronic check,70.70,151.65,Yes,1,Moved


### Some Basic Checks

In [10]:
# Datatypes of object
df.dtypes

CustomerID            object
Count                  int64
Country               object
State                 object
City                  object
Zip Code               int64
Lat Long              object
Latitude             float64
Longitude            float64
Gender                object
Senior Citizen        object
Partner               object
Dependents            object
Tenure Months          int64
Phone Service         object
Multiple Lines        object
Internet Service      object
Online Security       object
Online Backup         object
Device Protection     object
Tech Support          object
Streaming TV          object
Streaming Movies      object
Contract              object
Paperless Billing     object
Payment Method        object
Monthly Charges      float64
Total Charges         object
Churn Label           object
Churn Value            int64
Churn Reason          object
dtype: object

In [12]:
# How many rows and columns we have
df.shape

(7043, 31)

In [14]:
# Nulls and Datatypes
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7043 entries, 0 to 7042
Data columns (total 31 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   CustomerID         7043 non-null   object 
 1   Count              7043 non-null   int64  
 2   Country            7043 non-null   object 
 3   State              7043 non-null   object 
 4   City               7043 non-null   object 
 5   Zip Code           7043 non-null   int64  
 6   Lat Long           7043 non-null   object 
 7   Latitude           7043 non-null   float64
 8   Longitude          7043 non-null   float64
 9   Gender             7043 non-null   object 
 10  Senior Citizen     7043 non-null   object 
 11  Partner            7043 non-null   object 
 12  Dependents         7043 non-null   object 
 13  Tenure Months      7043 non-null   int64  
 14  Phone Service      7043 non-null   object 
 15  Multiple Lines     7043 non-null   object 
 16  Internet Service   7043 

In [16]:
# Summary Statistics
df.describe()

,Count,Zip Code,Latitude,Longitude,Tenure Months,Monthly Charges,Churn Value
count,7043.0,7043.000000,7043.000000,7043.000000,7043.000000,7043.000000,7043.000000
mean,1.0,93521.964646,36.282441,-119.798880,32.371149,64.761692,0.265370
std,0.0,1865.794555,2.455723,2.157889,24.559481,30.090047,0.441561
min,1.0,90001.000000,32.555828,-124.301372,0.000000,18.250000,0.000000
25%,1.0,92102.000000,34.030915,-121.815412,9.000000,35.500000,0.000000
50%,1.0,93552.000000,36.391777,-119.730885,29.000000,70.350000,0.000000
75%,1.0,95351.000000,38.224869,-118.043237,55.000000,89.850000,1.000000
max,1.0,96161.000000,41.962127,-114.192901,72.000000,118.750000,1.000000


#### Notes
* Multiple object columns
* Drop Columns --> Count (all just 1's) ; Country (United States) ; State (California) ; Churn Label (have Churn Value)
* Sanity Checks --> Lat Long is combination of the Latitude and Longitude columns ; How is Total Charges Calculated? 
* Nulls ---> Churn Reason (5174 - All the customers that haven't left yet) ; Spaces in Total charges (difficulty in converting to float)

### Feature Engineering

#### Basic Ones
* Mappinng Yes and No's to 1's and 0's
* Fixing Total Charges

In [58]:
yn_cols = [col for col in df.columns if ('Yes' and 'No') in list(df[f'{col}'].unique()) ]

In [84]:
# list of columns with yes's and no's
yn_cols = ['Senior Citizen','Partner','Dependents','Phone Service','Multiple Lines','Online Security',
             'Online Backup','Device Protection','Tech Support','Streaming TV','Streaming Movies',
             'Paperless Billing']
# mapping
for col in yn_cols:
    df[f'{col}'] = df[f'{col}'].map({'No':0, 'Yes':1, 'No phone service':0, 'No internet service':0})
    df[f'{col}'].astype(int)
    


In [86]:
# check thats all they have
for col in yn_cols:
    print(f'The column {col} contains these values {df[col].unique()}')

The column Senior Citizen contains these values [0 1]
The column Partner contains these values [0 1]
The column Dependents contains these values [0 1]
The column Phone Service contains these values [1 0]
The column Multiple Lines contains these values [0 1]
The column Online Security contains these values [1 0]
The column Online Backup contains these values [1 0]
The column Device Protection contains these values [0 1]
The column Tech Support contains these values [0 1]
The column Streaming TV contains these values [0 1]
The column Streaming Movies contains these values [0 1]
The column Paperless Billing contains these values [1 0]


In [88]:
df.head()

,CustomerID,Count,Country,State,City,Zip Code,Lat Long,Latitude,Longitude,Gender,Senior Citizen,Partner,Dependents,Tenure Months,Phone Service,Multiple Lines,Internet Service,Online Security,Online Backup,Device Protection,Tech Support,Streaming TV,Streaming Movies,Contract,Paperless Billing,Payment Method,Monthly Charges,Total Charges,Churn Label,Churn Value,Churn Reason
0,3668-QPYBK,1,United States,California,Los Angeles,90003,"33.964131, -118.272783",33.964131,-118.272783,Male,0,0,0,2,1,0,DSL,1,1,0,0,0,0,Month-to-month,1,Mailed check,53.85,108.15,Yes,1,Competitor made better offer
1,9237-HQITU,1,United States,California,Los Angeles,90005,"34.059281, -118.30742",34.059281,-118.307420,Female,0,0,1,2,1,0,Fiber optic,0,0,0,0,0,0,Month-to-month,1,Electronic check,70.70,151.65,Yes,1,Moved
2,9305-CDSKC,1,United States,California,Los Angeles,90006,"34.048013, -118.293953",34.048013,-118.293953,Female,0,0,1,8,1,1,Fiber optic,0,0,1,0,1,1,Month-to-month,1,Electronic check,99.65,820.5,Yes,1,Moved
3,7892-POOKP,1,United States,California,Los Angeles,90010,"34.062125, -118.315709",34.062125,-118.315709,Female,0,1,1,28,1,1,Fiber optic,0,0,1,1,1,1,Month-to-month,1,Electronic check,104.80,3046.05,Yes,1,Moved
4,0280-XJGEX,1,United States,California,Los Angeles,90015,"34.039224, -118.266293",34.039224,-118.266293,Male,0,0,1,49,1,1,Fiber optic,0,1,1,0,1,1,Month-to-month,1,Bank transfer (automatic),103.70,5036.3,Yes,1,Competitor had better devices


* So can map **['Senior Citizen','Partner','Dependents','Phone Service', 'Paperless Billing']**
* The others **['Multiple Lines','Online Security','Online Backup','Device Protection','Tech Support','Streaming TV','Streaming Movies',]** contain an extra _'No phone sevice'_ for **'Multiple Lines'** Column and _'No internet service'_ for the rest ---> carry out sanity checks /// also OHE or just map to 0?

In [28]:
list(df['Phone Service'].unique())

['Yes', 'No']

In [65]:
# Shows that Total Charges has nulls in the form of an empty string
(df['Total Charges'] == ' ').any()

True

In [67]:
# Gives the indexes for where this happens
df[df['Total Charges'] == ' '].index # happens when tenure month = 0

Index([2234, 2438, 2568, 2667, 2856, 4331, 4687, 5104, 5719, 6772, 6840], dtype='int64')

In [69]:
# Converts the column to float type with the empty string as nulls
df['Total Charges'] = pd.to_numeric(df['Total Charges'], errors='coerce')

# Fills nulls with zeros 
df['Total Charges'] = df['Total Charges'].fillna(value = 0)

In [71]:
df.head()

,CustomerID,Count,Country,State,City,Zip Code,Lat Long,Latitude,Longitude,Gender,Senior Citizen,Partner,Dependents,Tenure Months,Phone Service,Multiple Lines,Internet Service,Online Security,Online Backup,Device Protection,Tech Support,Streaming TV,Streaming Movies,Contract,Paperless Billing,Payment Method,Monthly Charges,Total Charges,Churn Label,Churn Value,Churn Reason
0,3668-QPYBK,1,United States,California,Los Angeles,90003,"33.964131, -118.272783",33.964131,-118.272783,Male,No,No,No,2,Yes,No,DSL,Yes,Yes,No,No,No,No,Month-to-month,Yes,Mailed check,53.85,108.15,Yes,1,Competitor made better offer
1,9237-HQITU,1,United States,California,Los Angeles,90005,"34.059281, -118.30742",34.059281,-118.307420,Female,No,No,Yes,2,Yes,No,Fiber optic,No,No,No,No,No,No,Month-to-month,Yes,Electronic check,70.70,151.65,Yes,1,Moved
2,9305-CDSKC,1,United States,California,Los Angeles,90006,"34.048013, -118.293953",34.048013,-118.293953,Female,No,No,Yes,8,Yes,Yes,Fiber optic,No,No,Yes,No,Yes,Yes,Month-to-month,Yes,Electronic check,99.65,820.50,Yes,1,Moved
3,7892-POOKP,1,United States,California,Los Angeles,90010,"34.062125, -118.315709",34.062125,-118.315709,Female,No,Yes,Yes,28,Yes,Yes,Fiber optic,No,No,Yes,Yes,Yes,Yes,Month-to-month,Yes,Electronic check,104.80,3046.05,Yes,1,Moved
4,0280-XJGEX,1,United States,California,Los Angeles,90015,"34.039224, -118.266293",34.039224,-118.266293,Male,No,No,Yes,49,Yes,Yes,Fiber optic,No,Yes,Yes,No,Yes,Yes,Month-to-month,Yes,Bank transfer (automatic),103.70,5036.30,Yes,1,Competitor had better devices


## EDA

In [ ]:
# for my_column in df.columns:
#     plt.figure(figsize=(8, 5))
#     sns.scatterplot(y=my_column, 
#                   x="City",
#                   hue="Churn Value", 
#                   data=df)
    
#     plt.title(f'Corrleation of {my_column} against City')
#     plt.xlabel(my_column)
#     plt.ylabel('City')
#     plt.xticks(rotation = 45,
#               horizontalalignment = 'right')
#     plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left')
#     #plt.grid()  # adds grid --> looks messy for the first graph
#     plt.show()

## More Feature Engineering